# Clustering
This notebook is to demonstrate the function of gathering the data from the stock market and creating the clustering algorithm.

In [4]:
# Import libraries
import yfinance as yf
import pandas as pd
import numpy as np
import json

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


In [5]:
class Clustering():
    '''

    Class to gather stock dataset and training the model.

    methods:
    - gather_data
    - cluster_export

    '''

    def __init__(self, stock_list):
        self.stock_list = stock_list
    
    def __get_increment(self, symbol):
        '''

        Get the increment of the stock in 1 month period.

        return: Float increment.

        '''

        stock_data = yf.download(
            tickers=symbol,
            period='1mo',
            interval='1wk'
        )

        stock_data = stock_data['Adj Close']

        latest_price = stock_data[-1]
        initial_price = stock_data[0]
        
        if initial_price is None:
            initial_price = stock_data[1]
        
        if latest_price is None:
            latest_price = stock_data[-2]

        increment = (latest_price - initial_price) / initial_price

        return increment

    def gather_data(self):
        '''

        Gathering and formatting stock data into a dataframe and export it to CSV file.

        return: None.

        '''
        list_columns = [
            'symbol',
            'sector',
            'fullTimeEmployees',
            'trailingAnnualDividendYield',
            'payoutRatio',
            'averageDailyVolume10Day',
            'trailingAnnualDividendRate',
            'averageVolume10days',
            'dividendRate',
            'beta',
            'priceHint',
            'trailingPE',
            'regularMarketVolume',
            'marketCap',
            'averageVolume',
            'priceToSalesTrailing12Months',
            'forwardPE',
            'fiveYearAvgDividendYield',
            'dividendYield',
            'enterpriseToRevenue',
            'profitMargins',
            'enterpriseToEbitda',
            'forwardEps',
            'bookValue',
            'sharesPercentSharesOut',
            'heldPercentInstitutions',
            'netIncomeToCommon',
            'trailingEps',
            'lastDividendValue',
            'priceToBook',
            'heldPercentInsiders',
            'shortRatio',
            'enterpriseValue',
            'earningsQuarterlyGrowth',
            'pegRatio',
            'shortPercentOfFloat',
            'increment'
        ]

        df = pd.DataFrame(columns=list_columns)
        for symbol in self.stock_list:
            try:
                print('[INFO] Gathering {} stock data...'.format(symbol))
                ticker = yf.Ticker(symbol)

                # Create DataFrame from Company Info
                company_info = ticker.info
                df_info = pd.DataFrame.from_dict(company_info, orient='index').T

                if df_info['fullTimeEmployees'] is None:
                    pass
                else:
                    df_info['symbol'] = [symbol]

                    # Get increment
                    increment = self.__get_increment(symbol)
                    df_info['increment'] = [increment]

                    # Remove unnecessary column
                    df_info.drop(df_info.columns.difference(list_columns), 1, inplace=True)

                    # Add to the main dataframe
                    df = df.append(df_info, ignore_index=True)
            except:
                pass

        # Export dataset
        export = df.to_csv('Stock Dataset.csv', index=False)
        print('[SUCCESS] Stock Dataset has been exported to the local directory.')

    def normalize_data(self, path):
        '''

        Normalize the data.

        @param path: Folder directory of the normalized dataset.

        return: None.

        '''

        df = pd.read_csv('Stock Dataset.csv')

        # Replace inf to nan and remove the row
        df = df.fillna(0)
        df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, inplace=True)

        # Index
        symbol_list = df['symbol'].values

        # Sector
        sector_list = df['sector'].values

        # Drop non digit columns
        df = df.drop(columns=['symbol', 'sector'])

        # Normalize
        df_values = df.values
        scaler = StandardScaler()
        df_values_scaled = scaler.fit_transform(df_values)
        df = pd.DataFrame(df_values_scaled, columns=list_columns[2:])

        # Readd the sector and symbol columns
        df['symbol'] = symbol_list
        df['sector'] = sector_list

        export = df.to_csv('Normalized Stock Dataset.csv', index=False)
        print('[SUCCESS] Normalized Stock Dataset has been exported to the local directory.')
        
    def cluster_export(self, path, index='symbol', n_clusters=500):
        '''

        Cluster the dataset and export to a new csv dataset with clustered value.

        @param path: Folder directory of the normalized dataset.
        @param index: Name of the column of index.
        @param n_clusters: Number of cluster

        return: None.

        '''

        df = pd.read_csv(path, index_col=index)

        model = KMeans(n_clusters=n_clusters)

        model = model.fit(df.values)

        # Cluster labels
        labels = model.labels_

        df['cluster'] = labels

        df = df['cluster']

        export = df.to_csv('Clustered Stock Dataset.csv')
        print('[SUCCESS] Clustered Stock Dataset has been exported to the local directory.')


In [3]:
with open('tickers.json', 'r') as f:
    json_tickers = f.read()
    list_tickers = json.loads(json_tickers)

test_tickers = ['AAPL', 'MSFT', 'VOO', 'AAAU', 'WNC']

clustering = Clustering(list_tickers)

# Run to gather the data
clustering.gather_data()

*************100%***********************]  1 of 1 completed
[INFO] Gathering WLTW stock data...
[*********************100%***********************]  1 of 1 completed
[INFO] Gathering WM stock data...
[*********************100%***********************]  1 of 1 completed
[INFO] Gathering WMB stock data...
[*********************100%***********************]  1 of 1 completed
[INFO] Gathering WMC stock data...
[INFO] Gathering WMG stock data...
[*********************100%***********************]  1 of 1 completed
[INFO] Gathering WMK stock data...
[*********************100%***********************]  1 of 1 completed
[INFO] Gathering WMS stock data...
[*********************100%***********************]  1 of 1 completed
[INFO] Gathering WMT stock data...
[*********************100%***********************]  1 of 1 completed
[INFO] Gathering WNC stock data...
[*********************100%***********************]  1 of 1 completed
[INFO] Gathering WNEB stock data...
[*********************100%***********

ValueError: Input contains infinity or a value too large for dtype('float64').

In [14]:
df = pd.read_csv('Stock Dataset.csv')
print(len(df))
print(df.head())

3
   fullTimeEmployees  trailingAnnualDividendYield  payoutRatio  \
0           0.590821                    -0.998939     0.316525   
1           0.817333                    -0.367474     1.035412   
2          -1.408154                     1.366413    -1.351937   

   averageDailyVolume10Day  trailingAnnualDividendRate  averageVolume10days  \
0                 1.368480                   -0.366565             1.368480   
1                -0.375294                    1.366170            -0.375294   
2                -0.993186                   -0.999605            -0.993186   

   dividendRate      beta  priceHint  trailingPE  ...  priceToBook  \
0     -0.377075 -0.150286        0.0    0.652424  ...     1.306679   
1      1.368944 -1.142667        0.0    0.760414  ...    -0.184892   
2     -0.991870  1.292953        0.0   -1.412839  ...    -1.121787   

   heldPercentInsiders  shortRatio  enterpriseValue  earningsQuarterlyGrowth  \
0            -0.702338   -0.748188         0.902358    

In [6]:
print(df['sector'].describe())

count                     64
unique                    11
top       Financial Services
freq                      12
Name: sector, dtype: object


In [136]:
df_clustered = pd.read_csv('Clustered Stock Dataset.csv')

print(df_clustered.head())


  symbol  cluster
0      A        0
1     AA        0
2    AAA        3
3   AAAU        3
4   AACG       11


In [151]:
symbol_cluster = df_clustered.loc[df_clustered['symbol'] == 'JPM']['cluster'].values[0]
print(symbol_cluster)


7


In [152]:
same_cluster = df_clustered.loc[df_clustered['cluster'] == symbol_cluster]
print(same_cluster)


     symbol  cluster
692     BAC        7
1181      C        7
1538  CMCSA        7
1723   CSCO        7
3574     HD        7
4094   INTC        7
4374    JNJ        7
4394    JPM        7
4534     KO        7
4863     MA        7
5171    MRK        7
5185     MS        7
5602    NVS        7
5774   ORCL        7
5966    PEP        7
5981    PFE        7
6011     PG        7
7932    UNH        7
8028      V        7
8311     VZ        7
